In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time 
import mediapipe as mp

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

In [3]:
def mediapipe_detection(image , model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) #COLOR CONVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_styled_landmarks(image, results):

       for face_landmarks in results.multi_face_landmarks:

              mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_contours_style())

In [5]:
def extract_keypoints(results):
    for face_landmarks in results.multi_face_landmarks or np.zeros(478*3).flatten():
        faceLandmarks = np.array([[res.x , res.y , res.z ] for res in face_landmarks.landmark]).flatten() if results.multi_face_landmarks else np.zeros(478*3)
        return np.concatenate([faceLandmarks])

In [6]:
#Path for exported data, numpy arrays
DATA_PATH = os.path.join('Head_Face_Mesh_Data')

#Actions that we try to detect 
actions = np.array(['tension','no tension','Head_down'])

#Thirty videos worth of data
no_sequences = 30

# Videos are going to be thirty frames in length
sequence_length = 30

In [7]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [8]:
cap = cv2.VideoCapture(1)
# with mp_face_mesh.FaceMesh()
with mp_face_mesh.FaceMesh(max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
    
    #Loop through actions
    for action in actions:
        #Loop through sequences
        for sequence in range(no_sequences):
            #Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                #Read feed
                ret, image = cap.read()

                image.flags.writeable = False
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                results = face_mesh.process(image)

                # Draw the face mesh annotations on the image.
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                if results.multi_face_landmarks:

                      for face_landmarks in results.multi_face_landmarks:
                        mp_drawing.draw_landmarks(
                            image=image,
                            landmark_list=face_landmarks,
                            connections=mp_face_mesh.FACEMESH_TESSELATION,
                            landmark_drawing_spec=None,
                            connection_drawing_spec=mp_drawing_styles
                            .get_default_face_mesh_tesselation_style())


                #Apply wait logic
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120,200),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    #Show to screen
                    cv2.imshow('EMOTION DETECTION DATA COLLECTION',image)
                    cv2.waitKey(2000)

                else:
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    #Show to screen
                    cv2.imshow('EMOTION DETECTION DATA COLLECTION', image)



                # New export keypoints
                keypoints= extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                if cv2.waitKey(5) & 0xFF == 27:
                    break

    cap.release()